<a href="https://colab.research.google.com/github/21Ovi/TensorFlow_/blob/main/06_NPL_in_TensorFlow_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in TensorFlow

NPL has the goal of deriving information out of natural language (could be sequences text to speech).

Another common term for NLP problems is sequence to sequence problem (seq2seq)

In [1]:
!nvidia-smi

Sun Aug 13 12:44:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/21Ovi/Helper-Functions/main/helper_functions.py

# Import series of helper function for noteebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-08-13 12:44:25--  https://raw.githubusercontent.com/21Ovi/Helper-Functions/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10116 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.88K  --.-KB/s    in 0s      

2023-08-13 12:44:25 (106 MB/s) - ‘helper_functions.py’ saved [10116/10116]



## Get a text dataset

The dataset we're going to using is Kaggle's introduction to NPL dataset (text samples of Tweets labelled as disaster or not disaster).

Resource : https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-08-13 12:44:29--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.170.128, 142.251.175.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.170.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   702KB/s    in 0.8s    

2023-08-13 12:44:30 (702 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in.

But I prefer to get visualize straigt away.

So another way to do this is to use pandas.

In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Suffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look loke?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # Create random indexes not higher than the total number of samples

for row in train_df[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text: 
@johndcgow heard this few days ago while driving and near crashed the car from laughing to much

---

Target: 0 (not real disaster)
Text: 
@olliebailey11 havnt you crashed ? ??

---

Target: 0 (not real disaster)
Text: 
Could anyone tell me of this here has crashed or just taking a very long time #Windows10 PLEASE! http://t.co/3FZIDHQrK3

---

Target: 1 (real disaster)
Text: 
Intact+MH370+Part+Lifts+Odds+Plane+Glided+Not+Crashed+Into+Sea http://t.co/MjTN3qbgOS via @YahooFinance#Hope for answers.

---

Target: 1 (real disaster)
Text: 
Pak Army Helicopter crashed in Mansehra.

---



### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
## Use train_train_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                             train_df_shuffled["target"].to_numpy(),
                                                                             test_size=0.1, # use 10% of training data for validation set
                                                                             random_state=42)

In [12]:
# Check lengths
len(train_sentences), len(train_labels),len(val_sentences), len(val_labels),

(6851, 6851, 762, 762)

In [13]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are few ways to do this, namely:
* `Tokenization`- direct mapping of token ( a token could be a workd or a character) to number.
* `Embedding` - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=max_length, # how long do you want your sequence to be
                                    pad_to_max_tokens=True)

In [16]:
len(train_sentences[0].split())

7

In [17]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
# Setup text vactorize variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g how many words from a tweet does model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street~ "
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)

In [22]:
print(f"Orignal text: \n {random_sentence}\
      n\nVectorized Version:")
text_vectorizer([random_sentence])

Orignal text: 
 #GrowingUpSpoiled going clay pigeon shooting and crying because of the 'aftershock'      n
Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,  104,    1, 9782,  617,    7, 1693,  152,    6,    2,  966,
           0,    0,    0,    0]])>

In [23]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represnted by a vector 100 long
* `input_length` = length of the sequence being passed to the embedding layer

In [24]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                            output_dim=128, # output shape
                             embeddings_initializer="uniform",
                            input_length=max_length # how long is each input
                            )

embedding

In [25]:
# get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Orignal text: \n {random_sentence}\
      \n\nEmbedding Version: ")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Orignal text: 
 ok peace I hope I fall off a cliff along with my dignity      

Embedding Version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03707414,  0.01420761,  0.02826861, ...,  0.03591237,
         -0.0286641 ,  0.04368428],
        [-0.02244141, -0.03720064,  0.01094451, ..., -0.03845116,
         -0.03285266,  0.03726684],
        [ 0.03948678,  0.00571388, -0.03087059, ..., -0.00608085,
         -0.02803742,  0.03266858],
        ...,
        [-0.01167087, -0.03243404,  0.03140627, ...,  0.03548861,
          0.03636532, -0.00323659],
        [-0.01878273, -0.03579617, -0.0146661 , ...,  0.03844342,
          0.01463515,  0.00614627],
        [-0.01878273, -0.03579617, -0.0146661 , ...,  0.03844342,
          0.01463515,  0.00614627]]], dtype=float32)>

In [26]:
# Checkout a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03707414,  0.01420761,  0.02826861,  0.04724897, -0.01103493,
        -0.03623753,  0.01464948, -0.03932837,  0.04443177, -0.02982625,
         0.0370407 , -0.04213542, -0.04039549, -0.04624186, -0.01729012,
         0.03991062,  0.02314958, -0.01594846,  0.03867734, -0.00569909,
         0.03250995,  0.00950237, -0.03952845,  0.0189181 , -0.0274709 ,
        -0.01824523,  0.015703  ,  0.00500791,  0.04744558,  0.00951337,
         0.04306004, -0.00221493,  0.01052064, -0.01946727,  0.0196332 ,
        -0.01303638, -0.01811736,  0.00103226,  0.03679463,  0.04698702,
        -0.01761396, -0.01249947, -0.03647034,  0.0261997 , -0.04148503,
        -0.04661047, -0.04058156, -0.03449357, -0.00757736, -0.0173061 ,
        -0.00174916, -0.04302311, -0.04664784,  0.02969838,  0.04932984,
         0.0433517 ,  0.00603883,  0.04170748, -0.01192099,  0.01829079,
        -0.03689231, -0.01912652,  0.03216609, -0.02838513,  0.04663506,
  

## Modelling a text dataset (running series of experiments)

No we've got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We've start with a baseline and move on from there

* `Model 0`: Naive Bayes (baseline), this is from SkLearn ML Map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* `Model 1`: Feed-forward neural network (dense model)
* `Model 2`: LSTM Model (RNN)
* `Model 3`: GRU Model (RNN)
* `Model 4`: Bidirectional-LSTM Model (RNN)
* `Model 5`: 1D Convolutional Neural Network (CNN)
* `Model 6`: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* `Model 7`: Same as model 6 with 10% of training data

How are we going to approch all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate a model


### Model 0: Getting a baseline

as with all machine learning modelling experiments, it's important to create  a baseline model so you've got b benchmark for future eperiments to build a model

To create a baseline, we'll yse SkLearn's multinominal Naive Bayes using the TensorFlow TF-IDF formula to convert our words to numbers

> 🔑 **Note:** It's common practice to use non-DL algorithm as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words into numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [28]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [29]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Create an evaluation function for our model experiments

We could evaluate all of our model's prediction with different metrics everytime, however, this will be cumbersome and could easily be fixed with a function

Let's create one to compare our model's prediction with truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

In [30]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weightted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                   "recall": model_recall,
                   "F1": model_f1 }
  return model_results

In [31]:
# Get basedline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1': 0.7862189758049549}

### Model 1: A Simple Dense Model

In [32]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [33]:
# Build model with the Funtional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # input are 1-dimentional string
x = text_vectorizer(inputs) ## turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs so use sigmoid activation function

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [35]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [36]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230813-124437
Epoch 1/5
215/215 [==============================] - 20s 62ms/step - loss: 0.6112 - accuracy: 0.7002 - val_loss: 0.5374 - val_accuracy: 0.7493
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4425 - accuracy: 0.8193 - val_loss: 0.4726 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3463 - accuracy: 0.8609 - val_loss: 0.4636 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2847 - accuracy: 0.8891 - val_loss: 0.4663 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2378 - accuracy: 0.9117 - val_loss: 0.4830 - val_accuracy: 0.7900


In [37]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4830 - accuracy: 0.7900


[0.48296058177948, 0.7900262475013733]

In [38]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [39]:
# Look at the single prediction
model_1_pred_probs[0]

array([0.3214087], dtype=float32)

In [40]:
# Look at the first 10 predictions
model_1_pred_probs[:10]

array([[0.3214087 ],
       [0.7683228 ],
       [0.9975078 ],
       [0.09663438],
       [0.10348583],
       [0.9295362 ],
       [0.91301906],
       [0.99302316],
       [0.9631132 ],
       [0.23373573]], dtype=float32)

In [41]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [42]:
# Caculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.00262467191601,
 'precision': 0.7964718621790533,
 'recall': 0.7900262467191601,
 'F1': 0.7866259949990327}

In [43]:
# Looks like our baseline model is out performing our first deep learning model
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False,  True])

## Visualizing learned embeddings

In [44]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [46]:
# Get the weeight matrix of embedding layer
# (these are the numerical represntation of each token in our training data, which have been learned for ~5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape # same size as vocab size and embedding_dim (output dim of our embedding layer)

(10000, 128)

In [47]:
embed_weights

array([[-0.00627469, -0.05041127, -0.00306264, ...,  0.03194254,
         0.00148847, -0.0078137 ],
       [-0.01135645, -0.00528777,  0.05463324, ..., -0.04482296,
        -0.00318995, -0.01142331],
       [ 0.05347889,  0.02556551,  0.01088328, ...,  0.04042603,
        -0.03242819,  0.00625483],
       ...,
       [-0.01236303,  0.01653055,  0.01946333, ..., -0.04947067,
        -0.03936294, -0.01918384],
       [ 0.03363997, -0.06816381,  0.07451656, ..., -0.01274014,
        -0.0180987 , -0.06028937],
       [ 0.02194184, -0.03999951,  0.0908412 , ..., -0.06520423,
        -0.06821558, -0.10823922]], dtype=float32)

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector : http://projector.tensorflow.org/

And TensorFlow also has an incredible guide on workds embeddings themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [48]:
# Create embedding filrs (we got this from TensorFlow's word embeddings documentations)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


In [49]:
# Download files from Colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN)

RNN's are useful for sequence data

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

### Model 2: LSTM

LSTM = Long Short Term Memory (one of the most popular LSTM cells)

Our structure of RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability
```

In [50]:
# Create an LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to return_sequences=True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [51]:
# Get a summay
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [52]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [53]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                      "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230813-124508
Epoch 1/5
215/215 [==============================] - 15s 53ms/step - loss: 0.2184 - accuracy: 0.9228 - val_loss: 0.5461 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1549 - accuracy: 0.9429 - val_loss: 0.6972 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1273 - accuracy: 0.9526 - val_loss: 0.7156 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1069 - accuracy: 0.9593 - val_loss: 0.8835 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0849 - accuracy: 0.9679 - val_loss: 0.9704 - val_accuracy: 0.7756


In [54]:
# Make prediction with LSTM model
model_2_preds_probs = model_2.predict(val_sentences)
model_2_preds_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.4237494e-03],
       [3.0313122e-01],
       [9.9957103e-01],
       [1.0088335e-02],
       [2.5850476e-04],
       [9.9400771e-01],
       [5.4926884e-01],
       [9.9962735e-01],
       [9.9937433e-01],
       [4.2515248e-01]], dtype=float32)

In [55]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_preds_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [56]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'precision': 0.7810725814480683,
 'recall': 0.7755905511811023,
 'F1': 0.7720383798735819}

## Model 3: GRU

Gated Recurrent Unit

Another poweful and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less paraments

In [57]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# print(x.shape)
# x = layers.GRU(64, return_sequences=True)(x) # if you want to stack the recurrent layers on the top of each others, you need return_sequence=True
# print(x.shape)
# x = layers.LSTM(42, return_sequences=True)(x)
# print(x.shape)
# x = layers.GRU(99)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [58]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [59]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [60]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230813-124532
Epoch 1/5
215/215 [==============================] - 13s 50ms/step - loss: 0.1588 - accuracy: 0.9337 - val_loss: 0.7136 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0827 - accuracy: 0.9675 - val_loss: 0.8287 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0695 - accuracy: 0.9717 - val_loss: 0.8549 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0572 - accuracy: 0.9755 - val_loss: 1.0027 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0510 - accuracy: 0.9780 - val_loss: 1.2042 - val_accuracy: 0.7769


In [61]:
# Make some prediction with ouyt GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[4.1915569e-04],
       [8.2492501e-01],
       [9.9989390e-01],
       [4.7008682e-02],
       [7.0135851e-05],
       [9.9932611e-01],
       [1.8902963e-01],
       [9.9995530e-01],
       [9.9989438e-01],
       [9.3174404e-01]], dtype=float32)

In [62]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [63]:
# Calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.69028871391076,
 'precision': 0.7817868897568192,
 'recall': 0.7769028871391076,
 'F1': 0.7736103146611698}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an english sentence) however, a bidirectional RNN goes from right to left as well as left to right

In [64]:
# Build a bidirectional RNN in TensorFlow
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
# print(x.shape)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [65]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [66]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [67]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230813-124555
Epoch 1/5
215/215 [==============================] - 15s 52ms/step - loss: 0.1128 - accuracy: 0.9644 - val_loss: 0.8622 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0551 - accuracy: 0.9772 - val_loss: 1.2372 - val_accuracy: 0.7598
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0455 - accuracy: 0.9797 - val_loss: 1.4358 - val_accuracy: 0.7533
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0446 - accuracy: 0.9781 - val_loss: 1.4481 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0375 - accuracy: 0.9812 - val_loss: 1.5494 - val_accuracy: 0.7585


In [68]:
# Make predictions with out bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


array([[6.6107436e-04],
       [9.9317497e-01],
       [9.9999011e-01],
       [1.8217687e-01],
       [2.9091429e-05],
       [9.9970347e-01],
       [9.4918633e-01],
       [9.9999332e-01],
       [9.9998951e-01],
       [9.9910933e-01]], dtype=float32)

In [69]:
# Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [70]:
# Calculate the results of our bidirectional model
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 75.8530183727034,
 'precision': 0.7583857376990771,
 'recall': 0.7585301837270341,
 'F1': 0.75741871177066}

## Convolutional Neral network for Text (and other types of sequences)

We've used CNNs for images are typically 2D (height x xwidth)... however, our text data is 1D

Previously we've Conv2D for our image data but now we're going to use Conv1D

The typicall structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (Typically Conv1D + pooling) -> Outputs (class probabilities)
```

In [71]:
### Model 5: Conv1D

In [72]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # this is also referred to as an ngram of 5 (meaning it looks at 5 words at a time)
                        strides=1, # default = "valid", the output is smaller then the input shape, "same" means output is same shape as input
                        activation="relu",
                        padding="valid") # default

conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get most important feature" or "get the feature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [73]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.05978124, -0.03567647, -0.00213423, ..., -0.01139026,
         -0.03128298, -0.03815832],
        [-0.01004015,  0.02685425,  0.0204581 , ...,  0.03462369,
         -0.02362236,  0.00552507],
        [-0.05179512, -0.06483479,  0.00110517, ...,  0.03058026,
         -0.03759419, -0.03375958],
        ...,
        [-0.01068546, -0.01792229, -0.00639625, ..., -0.01020053,
         -0.01461663,  0.0024677 ],
        [-0.01068546, -0.01792229, -0.00639625, ..., -0.01020053,
         -0.01461663,  0.0024677 ],
        [-0.01068546, -0.01792229, -0.00639625, ..., -0.01020053,
         -0.01461663,  0.0024677 ]]], dtype=float32)>

In [74]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.05398487, 0.        , 0.        , 0.        , 0.0534848 ,
         0.08749045, 0.0112723 , 0.00655358, 0.03721126, 0.00245472,
         0.03993552, 0.04726561, 0.01610658, 0.04786108, 0.        ,
         0.        , 0.        , 0.01068524, 0.01623221, 0.0293778 ,
         0.        , 0.        , 0.        , 0.01583272, 0.        ,
         0.04948774, 0.        , 0.04666599, 0.10747862, 0.00174021,
         0.        , 0.03404097],
        [0.10194897, 0.        , 0.02728792, 0.02488775, 0.03985922,
         0.01148424, 0.04842395, 0.        , 0.        , 0.        ,
         0.03183515, 0.02968451, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.04583715, 0.        ,
         0.03460366, 0.        , 0.02035271, 0.06109255, 0.        ,
         0.10472501, 0.02216221, 0.01798059, 0.        , 0.0325871 ,
         0.        , 0.        ],
        [0.03835203, 0.00034333, 0.06759606, 0.    

In [75]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.10194897, 0.03211282, 0.0707903 , 0.0277684 , 0.0534848 ,
        0.08749045, 0.04842395, 0.03406161, 0.03721126, 0.00245472,
        0.03994451, 0.08187188, 0.08737335, 0.04786108, 0.028513  ,
        0.0382691 , 0.        , 0.01674726, 0.04583715, 0.06810275,
        0.03541231, 0.0568383 , 0.039404  , 0.06109255, 0.06040863,
        0.10472501, 0.02216221, 0.04666599, 0.10747862, 0.0325871 ,
        0.        , 0.07650555]], dtype=float32)>

In [76]:
# Create 1-dimesional convolutional layer to model sequences
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [77]:
# Compile Conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [78]:
# Get a summary of our Conv1d model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [79]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230813-124644
Epoch 1/5
215/215 [==============================] - 14s 50ms/step - loss: 0.1244 - accuracy: 0.9588 - val_loss: 0.9582 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0759 - accuracy: 0.9714 - val_loss: 1.0611 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0631 - accuracy: 0.9747 - val_loss: 1.1459 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0557 - accuracy: 0.9774 - val_loss: 1.1778 - val_accuracy: 0.7546
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0505 - accuracy: 0.9788 - val_loss: 1.2973 - val_accuracy: 0.7598


In [80]:
# Make some predictions
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.4828025e-01],
       [8.2161671e-01],
       [9.9960917e-01],
       [8.4092453e-02],
       [8.5801382e-08],
       [9.8530662e-01],
       [8.8327497e-01],
       [9.9997282e-01],
       [9.9999535e-01],
       [9.3216348e-01]], dtype=float32)

In [81]:
# Convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [82]:
# Evaluate model 5 predictions
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)

model_5_results

{'accuracy': 75.98425196850394,
 'precision': 0.7620917281539858,
 'recall': 0.7598425196850394,
 'F1': 0.7571687002970852}

## Model 6: TensorFlowHub PreTrained Feature Extractor

Now we've built a few of our own models, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder : https://tfhub.dev/google/universal-sentence-encoder/4

In [83]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                       "When you can the universal sentence encoder on a sentence, it turnes into numbers"])

print(embed_samples[0][:50])

tf.Tensor(
[-0.00477086  0.0293698   0.06304868 -0.00976107  0.03536275  0.05988206
  0.04076201  0.05354147 -0.00170396  0.01987924  0.01672826  0.00499968
  0.02665901  0.08346527  0.07317044 -0.05286197  0.03112919 -0.055748
  0.03831363 -0.0400802  -0.03261067  0.04829443  0.04296905  0.02417224
 -0.01937103 -0.01896992 -0.01253994  0.02908366 -0.04963567 -0.02397985
 -0.05901828  0.01279296 -0.02139725 -0.03211551 -0.00769663 -0.06545766
  0.00214907  0.04122009 -0.01750335 -0.07401229 -0.00822927 -0.0333032
 -0.0265524   0.05162273 -0.11012802 -0.01490786 -0.01965857  0.00943312
 -0.02796747  0.03919587], shape=(50,), dtype=float32)


In [84]:
embed_samples[0].shape

TensorShape([512])

In [85]:
sample_sentence

"There's a flood in my street~ "

In [86]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [87]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

In [88]:
# Compile the model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [89]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 64)                32832     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [90]:
# Train a classifier on top of USE pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=6,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentences_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentences_encoder/20230813-124803
Epoch 1/6
215/215 [==============================] - 6s 12ms/step - loss: 0.5031 - accuracy: 0.7847 - val_loss: 0.4466 - val_accuracy: 0.8005
Epoch 2/6
215/215 [==============================] - 2s 11ms/step - loss: 0.4135 - accuracy: 0.8173 - val_loss: 0.4376 - val_accuracy: 0.8097
Epoch 3/6
215/215 [==============================] - 2s 10ms/step - loss: 0.4007 - accuracy: 0.8235 - val_loss: 0.4312 - val_accuracy: 0.8123
Epoch 4/6
215/215 [==============================] - 2s 10ms/step - loss: 0.3915 - accuracy: 0.8254 - val_loss: 0.4312 - val_accuracy: 0.8110
Epoch 5/6
215/215 [==============================] - 2s 10ms/step - loss: 0.3847 - accuracy: 0.8286 - val_loss: 0.4237 - val_accuracy: 0.8150
Epoch 6/6
215/215 [==============================] - 2s 10ms/step - loss: 0.3775 - accuracy: 0.8356 - val_loss: 0.4248 - val_accuracy: 0.8215


In [91]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 8ms/step


array([[0.18129757],
       [0.7763289 ],
       [0.9900003 ],
       [0.2000551 ],
       [0.6764708 ],
       [0.6959849 ],
       [0.9785791 ],
       [0.9833543 ],
       [0.9425969 ],
       [0.08536042]], dtype=float32)

In [92]:
# Convert prediction probabilites to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [93]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 82.1522309711286,
 'precision': 0.823814157518236,
 'recall': 0.821522309711286,
 'F1': 0.8200994568813982}

In [94]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1': 0.7862189758049549}

## Model 7: TF Hub Pretrained USE but with 10% of trainin data

Transfer Learning really helps when you don't have a large dataset.

To see how our model performs on a smaller dataset, let's replicate `model_6` except we'll train it on 10% of the data

In [106]:
# ## NOTE: Making data splits like below leads to data leakage (model_7 trained on 10% data, outperforms model_6 train on 100% of data)
# ## DO NOT MAKE DATA SPLITS WHICH LEAK DATA FROM VALIDATION/TEST SETS INTO TRAINING SETS

# # Create subsets of 10% of the training data
# train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)

# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()

# len(train_sentences_10_percent), len(train_labels_10_percent)

> 🔑 **Note:** Be *very* careful creating training/val/test splits that you don't leak data accross the datasets, otherwise your model evaluation metrics will be wrong. If something looks too good to be true (a model trained on 10% of data outperforming the same model trained on 100% of data) trust your gut and go back through to find where the error may lie.

In [111]:
## Making a better dataset split (no data leakage)
train_10_percent_split = int(0.1 * len(train_sentences))
train_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [114]:
# Check the number of target in our updated subset of data
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

In [115]:
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [116]:
# Create model using the Sequential API
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

In [117]:
# Compile the model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [118]:
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 64)                32832     
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [123]:
# Train a classifier on top of USE pretrained embeddings
model_7_history = model_7.fit(train_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_senteces_encoder_10_percent_correct_split")])

Saving TensorBoard log files to: model_logs/tf_hub_senteces_encoder_10_percent_correct_split/20230813-141029
Epoch 1/5
22/22 [==============================] - 3s 39ms/step - loss: 0.6628 - accuracy: 0.7080 - val_loss: 0.6396 - val_accuracy: 0.7507
Epoch 2/5
22/22 [==============================] - 0s 19ms/step - loss: 0.5916 - accuracy: 0.7942 - val_loss: 0.5811 - val_accuracy: 0.7717
Epoch 3/5
22/22 [==============================] - 0s 18ms/step - loss: 0.5161 - accuracy: 0.8131 - val_loss: 0.5303 - val_accuracy: 0.7717
Epoch 4/5
22/22 [==============================] - 0s 19ms/step - loss: 0.4576 - accuracy: 0.8248 - val_loss: 0.5015 - val_accuracy: 0.7782
Epoch 5/5
22/22 [==============================] - 0s 19ms/step - loss: 0.4190 - accuracy: 0.8336 - val_loss: 0.4877 - val_accuracy: 0.7808


In [124]:
# Make predictions with USE TF Hub model 10 percent
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 0s 7ms/step


array([[0.2091824 ],
       [0.59652644],
       [0.91252047],
       [0.39632365],
       [0.55681705],
       [0.69936687],
       [0.8857373 ],
       [0.82043   ],
       [0.86337876],
       [0.14168273]], dtype=float32)

In [125]:
# Convert prediction probabilities to labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [126]:
# Evaluate model 7 predictions
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)

model_7_results

{'accuracy': 78.08398950131233,
 'precision': 0.781027063798433,
 'recall': 0.7808398950131233,
 'F1': 0.7797836213601502}